In [1]:
import seaborn as sns

In [2]:
df=sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
#Handling missing values
#handling categorical features
#handling feature scaling and outliers
#automate the entire process

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [6]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
encoder=LabelEncoder()

In [10]:
df['time']=encoder.fit_transform(df['time'])

In [11]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [12]:
df.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2
243,18.78,3.00,Female,No,Thur,0,2


In [13]:
df['time'].value_counts()

0    176
1     68
Name: time, dtype: int64

## Divide the dependent and independent features

In [14]:
x=df.drop(columns='time',axis=1)

In [16]:
y=df['time']

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [22]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((195, 6), (49, 6), (195,), (49,))

In [23]:
# Handling missing values
from sklearn.impute import SimpleImputer
# handle the categorical values
from sklearn.preprocessing import OneHotEncoder
# feature scaling
from sklearn.preprocessing import StandardScaler
# we have to create automate pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    int32   
 6   size        244 non-null    int64   
dtypes: category(3), float64(2), int32(1), int64(1)
memory usage: 8.0 KB


In [25]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

## Feature Engineering Automation

In [27]:
num_pipeline=Pipeline(
                     steps=[
                         ('imputer',SimpleImputer(strategy='median')),#handling missing values
                         ('scaler',StandardScaler())#feature scaling
                     ]
)

cat_pipeline=Pipeline(
                    steps=[
                        ('imputer',SimpleImputer(strategy='most_frequent')),#handling missing values
                        ('onehotencoder',OneHotEncoder())#categorical feature to numerical
                    ]
)

## Create a rapper of those two

In [30]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [31]:
x_train=preprocessor.fit_transform(x_train)

In [32]:
x_test=preprocessor.transform(x_test)

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB


In [56]:
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'Naive Bayes':GaussianNB()
}

In [57]:
from sklearn.metrics import accuracy_score

In [58]:
def evaluate_model(x_train,y_train,x_test,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(x_train,y_train)
        y_pred_test=model.predict(x_test)
        test_model_score=accuracy_score(y_test,y_pred_test)
        report[list(models.values())[i]]=test_model_score
    return report


evaluate_model(x_train,y_train,x_test,y_test,models)

{RandomForestClassifier(): 0.9591836734693877,
 DecisionTreeClassifier(): 0.9387755102040817,
 GaussianNB(): 0.9591836734693877}

In [40]:
classfier=RandomForestClassifier()


## HyperParameter Tuning

In [41]:
params={
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [42]:
from sklearn.model_selection import RandomizedSearchCV

In [44]:
cv=RandomizedSearchCV(classfier,param_distributions=params,cv=5,scoring='accuracy',verbose=3)

In [45]:
cv

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [46]:
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.897 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.923 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=1.000 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.897 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.949 total time=  

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [59]:
y_pred=cv.predict(x_test)

In [60]:
cv.best_params_

{'n_estimators': 300, 'max_depth': None, 'criterion': 'entropy'}

In [61]:
score=accuracy_score(y_test,y_pred)

In [62]:

score

0.9591836734693877